In [1]:
# 1. 필요한 라이브러리 설치 (가상 환경이 활성화된 상태에서)
!pip install langchain langchain-community langchain-core
!pip install transformers torch  # 또는 tensorflow (선호하는 딥러닝 프레임워크에 따라)
!pip install pytorch-lightning
!pip install datasets
!pip install scikit-learn
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.5 MB/s eta 0:00:00a 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 11.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 5.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 7.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.2/444.2 kB 8.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.2/466.2 kB 7.3 MB/s eta 0:00:0000:01
  Using cached packaging-24.2-py3-none-any.whl (65 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl (15 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 3.5 MB/s eta 0:00:00
  Using cached aiosignal-1.3.

In [1]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import os
from torch.utils.data import Dataset, DataLoader
import datasets
from datasets import load_dataset
import pandas as pd
import pytorch_lightning as pl
import torch
torch.set_float32_matmul_precision('medium')
import torch.nn as nn
import glob
from transformers import ElectraModel, AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split

In [2]:
###########################
# KOTE 파인튜닝 모델 로드 (체크포인트 파일 경로 수정)
###########################
best_ckpt_path_kote = './model/250127_KcElectra_kote.ckpt' # Colab 경로에 맞게 수정
print("Best checkpoint path (KOTE Finetuned):", best_ckpt_path_kote)

# KOTETagger 클래스는 이전 코드와 동일
class KOTETagger(pl.LightningModule): # KOTETagger 클래스 정의 (이전 코드에서 복사)
    def __init__(self, n_training_steps=None, n_warmup_steps=None):
        super().__init__()
        self.electra = AutoModel.from_pretrained(MODEL_NAME, return_dict=True) # pretrained_electra 제거 및 직접 로드
        self.classifier = nn.Linear(self.electra.config.hidden_size, 44) # num_labels=44 (KOTE 라벨 개수)
        self.n_training_steps = n_training_steps
        self.n_warmup_steps = n_warmup_steps
        self.criterion = nn.BCELoss()

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.electra(input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :]
        logits = self.classifier(cls_output)
        probs = torch.sigmoid(logits)
        loss = 0
        if labels is not None:
            loss = self.criterion(probs, labels)
        return loss, probs

    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, _ = self(input_ids, attention_mask, labels)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        return {"val_loss": loss}

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=INITIAL_LR, weight_decay=WEIGHT_DECAY) # 가중치 감쇠
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=self.n_warmup_steps,
            num_training_steps=self.n_training_steps
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val_loss",
                "interval": "step",
                "frequency": 1
            }
        }

Best checkpoint path (KOTE Finetuned): ./model/250127_KcElectra_kote.ckpt


In [3]:

###########################
# 커스텀 Dataset 정의 (PoetryDataset) - 가중치 제거 (가중치 미사용)
###########################
MAX_LENGTH = 512

class PoetryDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=MAX_LENGTH, use_data_augmentation=False): # weights 제거
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.use_data_augmentation = use_data_augmentation

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        labels = self.labels[idx]

        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
            return_token_type_ids=False,
        )

        # if self.use_data_augmentation: # 데이터 증강 OFF
        #     encoding = mask_and_switch(encoding, prob=0.1)

        return dict(
            input_ids=encoding["input_ids"].squeeze(),
            attention_mask=encoding["attention_mask"].squeeze(),
            labels=torch.FloatTensor(labels), # weights 제거
        )

In [4]:
###########################
# DataModule 정의 (PoetryDataModule)
###########################
class PoetryDataModule(pl.LightningDataModule):
    def __init__(self, train_dataset, val_dataset, test_dataset, batch_size=16, num_workers=0): # num_workers=0 for Colab
        super().__init__()
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset
        self.test_dataset = test_dataset
        self.batch_size = batch_size
        self.num_workers = num_workers

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.num_workers
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers
        )

    def test_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers
        )

In [5]:
###########################
# 4. KOTE 모델 정의 (동일)
###########################

device = "cuda" if torch.cuda.is_available() else "cpu"

class KOTEtagger_(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.electra = ElectraModel.from_pretrained("beomi/KcELECTRA-base", revision='v2021').to(device)
        self.tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base", revision='v2021')
        self.classifier = nn.Linear(self.electra.config.hidden_size, 44).to(device)
        
    def forward(self, text:str):
        encoding = self.tokenizer.encode_plus(
          text,
          add_special_tokens=True,
          max_length=512,
          return_token_type_ids=False,
          padding="max_length",
          return_attention_mask=True,
          return_tensors='pt',
        ).to(device)
        output = self.electra(encoding["input_ids"], attention_mask=encoding["attention_mask"])
        output = output.last_hidden_state[:,0,:]
        output = self.classifier(output)
        output = torch.sigmoid(output)
        torch.cuda.empty_cache()
        
        return output

In [6]:
###########################
# 근현대시 데이터셋 로드 및 전처리 (기존 코드 활용 + 일치 라벨만 사용)
############################ 데이터프레임으로 불러오기 (실제 파일 경로로 수정)
df = pd.read_csv("../data/총합데이터셋_0601_5인 - 행단위.csv") # Colab 경로에 맞게 수정

# 감정 라벨 데이터를 리스트로 변환하는 함수
def labels_to_list(labels_str):
    if pd.isna(labels_str):
        return []
    return [label.strip() for label in labels_str.split(',')]

# 라벨 데이터를 리스트로 변환
df['annotator01_label_list'] = df['annotator01'].apply(labels_to_list)
df['annotator02_label_list'] = df['annotator02'].apply(labels_to_list)
df['annotator03_label_list'] = df['annotator03'].apply(labels_to_list)
df['annotator04_label_list'] = df['annotator04'].apply(labels_to_list)
df['annotator05_label_list'] = df['annotator05'].apply(labels_to_list)

In [7]:
def get_labels_agreed_by_at_least_k(row, k=3):
    """
    각 행(row)에 대해, 최소 k명 이상이 동의한 감정만 추출

    Parameters:
    - row: annotator label list들이 있는 DataFrame row
    - k: 동의한 annotator 최소 수 (기본 2명)

    Returns:
    - 감정 리스트 중 k명 이상이 공통으로 선택한 감정 리스트
    """
    all_labels = (
        row['annotator01_label_list'] +
        row['annotator02_label_list'] +
        row['annotator03_label_list'] +
        row['annotator04_label_list'] +
        row['annotator05_label_list']
    )
    counter = pd.Series(all_labels).value_counts() # 감정별 개수 세기
    return [label for label, count in counter.items() if count >= k] # k명 이상이 동의한 감정 리스트

In [8]:
# 2명 이상 동의한 감정 리스트로 새 컬럼 생성
df['common_labels'] = df.apply(lambda row: get_labels_agreed_by_at_least_k(row, k=3), axis=1)

df[['common_labels']].head(10) # 3명 이상 동의한 감정 리스트 확인

,common_labels
0,[비장함]
1,"[비장함, 부끄러움]"
2,[기대감]
3,"[패배/자기혐오, 절망, 슬픔, 힘듦/지침]"
4,"[기쁨, 기대감, 아껴주는]"
5,"[비장함, 불쌍함/연민, 아껴주는]"
6,[]
7,[비장함]
8,"[기대감, 감동/감탄]"
9,[슬픔]


In [9]:
# 일치하는 라벨만 있는 데이터만 필터링
df_agreement = df[df['common_labels'].map(len) > 0].reset_index(drop=True) # agreement 컬럼이 비어있지 않은 행만 선택

# 검증용으로 사용할 "불일치" 데이터 분리 (전체 데이터 중 30%를 validation으로 사용)
# df_disagreement_for_val = df[df['agreement'].map(len) == 0].reset_index(drop=True)
# val_size_disagreement = int(0.3 * len(df_disagreement_for_val))
# df_val = df_disagreement_for_val.iloc[:val_size_disagreement]

# 1차 데이터 csv 파일에서 'agreement' 컬럼이 비어 있지 않은 행만 선택
df_agreement = df[df['common_labels'].apply(lambda x: len(x) > 0)]

# 'agreement' 컬럼의 리스트들을 새로운 'labels' 컬럼에 할당
df_agreement['labels'] = df_agreement['common_labels']
df_agreement_reset = df_agreement.reset_index()

#  cleaned labels가 비어 있지 않은 행만 필터링 - Define df_agreement_cleaned FIRST
df_agreement_cleaned = df_agreement_reset[df_agreement_reset['labels'].map(len) > 0].reset_index(drop = True) # Line 46 (moved up) - df_agreement_cleaned is DEFINED here FIRST

# 불용 라벨 제거 (optional): ['없음', 'nan', '', None] 라벨 제거
labels_to_remove = ['nan', '', None]


/var/folders/cd/mj8pxhhd6ml7kpdhggf19l_80000gn/T/ipykernel_69717/454827291.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_agreement['labels'] = df_agreement['common_labels']


In [10]:
def remove_labels(labels):
    return [label for label in labels if label not in labels_to_remove and pd.notna(label) and label != 'nan']

# Assign 'labels_cleaned' column to the ALREADY DEFINED df_agreement_cleaned
df_agreement_cleaned['labels_cleaned'] = df_agreement_reset['labels'].apply(remove_labels) # Line 43 (moved down) - Assign to df_agreement_cleaned AFTER it's defined

In [11]:
###########################
# 시드 고정
###########################
RANDOM_SEED = 42
pl.seed_everything(RANDOM_SEED, workers=True)

###########################
# 토크나이저 로드
###########################
MODEL_NAME = "beomi/KcELECTRA-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Seed set to 42


In [12]:
# # 검증 및 테스트 데이터 분리 (train:validation:test = 8:1:1)
train_texts_poetry, temp_texts, train_labels_poetry_raw, temp_labels_poetry_raw = train_test_split(
    df_agreement_cleaned['본문'].astype(str).tolist(),
    df_agreement_cleaned['labels_cleaned'].tolist(),
    test_size=0.2,
    random_state=RANDOM_SEED
)

val_texts_poetry, test_texts_poetry, val_labels_poetry_raw, test_labels_poetry_raw = train_test_split(
    temp_texts,
    temp_labels_poetry_raw,
    test_size=0.5,
    random_state=RANDOM_SEED
)

In [13]:
###########################
# Multi-Label Binarizer (근현대시 라벨 적용 및 재학습) (기존 코드 활용)
###########################
from sklearn.preprocessing import MultiLabelBinarizer

mlb_poetry = MultiLabelBinarizer()
train_labels_poetry = mlb_poetry.fit_transform(train_labels_poetry_raw)
val_labels_poetry = mlb_poetry.transform(val_labels_poetry_raw)
test_labels_poetry = mlb_poetry.transform(test_labels_poetry_raw)
LABELS = mlb_poetry.classes_.tolist()

# Dataset 인스턴스 생성 (PoetryDataset 사용) (기존 코드 활용 + weights 전달)
train_dataset_poetry = PoetryDataset(train_texts_poetry, train_labels_poetry, tokenizer, use_data_augmentation=False) # 가중치 제거
val_dataset_poetry = PoetryDataset(val_texts_poetry, val_labels_poetry, tokenizer, use_data_augmentation=False) # 가중치 제거
test_dataset_poetry = PoetryDataset(test_texts_poetry, test_labels_poetry, tokenizer, use_data_augmentation=False) # 가중치 제거

BATCH_SIZE = 32 # 배치 사이즈 감소 (원래 64, 32, 16, 8)
data_module_poetry = PoetryDataModule(
    train_dataset_poetry,
    val_dataset_poetry,
    test_dataset_poetry,
    batch_size=BATCH_SIZE,
    num_workers=0 # num_workers=0 for Colab
)

In [14]:
THRESHOLD = 0.3
sample_text = """하루 종일 지친 몸으로만 떠돌다가
땅에 떨어져 죽지 못한
햇빛들은 줄지어 어디로 가는 걸까

웅성웅성 가장 근심스러운 색깔로 서행하며
이미 어둠이 깔리는 소각장으로 몰려들어
몇 점 폐휴지로 타들어가는 오루 6시의 참혹한 형량
단 한 번 후회도 용서하지 않는 무서운 시간
바람은 긴 채찍을 휘둘러
살아서 빛나는 온갖 상징을 몰아내고 있다.

도시는 곧 활자들이 일제히 빠져 달아나
속도 없이 페이지를 펄럭이는 텅 빈 한 권 책이 되리라.
승부를 알 수 없는 하루와의 싸움에서
우리는 패배했을까. 오늘도 물어보는 사소한 물음은
그러나 우리의 일생을 텅텅 흔드는 것.

오후 6시의 소각장 위로 말없이
검은 연기가 우산처럼 펼쳐지고
이젠 우리들의 차례였다.
두렵지 않은가.
밤이면 그림자를 빼앗겨 누구나 아득한 혼자였다.

문득 거리를 빠르게 스쳐가는 일상의 공포
보여다오. 지금까지 무엇을 했는가 살아 있는 그대여
오후 6시 우리들 이마에도 아, 붉은 노을이 떴다.

그러면 우리는 어디로 가지?
아직도 펄펄 살아 있는 우리는 이제 각자 어디로 가지?
""" # 기형도 - 노을

In [15]:
# LightningModule 인스턴스 생성
kote_model = KOTEtagger_()
kote_model.load_state_dict(torch.load("./model/kote_pytorch_lightning.bin"), strict=False) # <All keys matched successfully>라는 결과가 나오는지 확인!

preds = kote_model(sample_text)[0]

for l, p in zip(LABELS, preds):
    if p>0.4:
        print(f"{l}: {p}")

기대감: 0.8426862359046936
부끄러움: 0.610651969909668
부담/안_내킴: 0.6299067735671997
슬픔: 0.6140410304069519
신기함/관심: 0.7828693389892578
의심/불신: 0.7446609139442444
패배/자기혐오: 0.4017426371574402
한심함: 0.5035011172294617
환영/호의: 0.9441425204277039


In [16]:
kote_finetuned_model = KOTETagger.load_from_checkpoint(best_ckpt_path_kote)
pretrained_electra = kote_finetuned_model.electra # 수정: electra backbone만 가져옴

In [17]:
###########################
# LightningModule 정의 (PoetryTagger) (기존 코드 활용 + 가중치 손실 함수, Dropout, Weight Decay, Learning Rate 감소, EarlyStopping patience 증가)
###########################
INITIAL_LR = 1e-5 # 학습률 감소 (원래 2e-5, 1e-5, 5e-6, 2e-6)
DROPOUT_RATE = 0.5 # Dropout 비율 (0.1, 0.3, 0.5) - Dropout 추가
WEIGHT_DECAY = 0.02 # Weight Decay 값 (0.001, 0.01, 0.02) - Weight Decay 추가

class PoetryTagger(pl.LightningModule):
    def __init__(self, n_training_steps=None, n_warmup_steps=None, dropout_rate=DROPOUT_RATE): # dropout_rate hyperparameter
        super().__init__()
        self.electra = pretrained_electra # 수정: KOTE 파인튜닝 모델의 electra backbone 사용
        self.classifier = nn.Sequential( # nn.Sequential 사용하여 dropout layer 추가
            nn.Linear(self.electra.config.hidden_size, len(LABELS)),
            nn.Dropout(dropout_rate)
        ) # Classifier 출력층 크기 자동 조정
        self.n_training_steps = n_training_steps
        self.n_warmup_steps = n_warmup_steps
        self.criterion = nn.BCELoss() # 기본 BCE Loss (가중치 미적용)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.electra(input_ids, attention_mask=attention_mask)
        # [CLS] 토큰 기준으로 분류
        cls_output = outputs.last_hidden_state[:, 0, :]
        logits = self.classifier(cls_output)
        probs = torch.sigmoid(logits)

        loss = 0
        if labels is not None:
            loss = self.criterion(probs, labels)

        return loss, probs

    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]

        loss, _ = self(input_ids, attention_mask, labels) # forward 함수에 weights 제거

        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]

        loss, outputs = self(input_ids, attention_mask, labels) # validation loss는 기존 BCE Loss 사용 (optional)
        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        return {"val_loss": loss} # validation metrics are optional

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=INITIAL_LR, weight_decay=WEIGHT_DECAY) # 가중치 감쇠
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=self.n_warmup_steps,
            num_training_steps=self.n_training_steps
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val_loss",
                "interval": "step",
                "frequency": 1
            }
        }

In [18]:
# ###########################
# # Best checkpoint load & Evaluation (poetry-finetuning) (기존 코드 활용)
# ###########################
def get_latest_version_dir_poetry(base_dir="./lightning_logs/poetry-finetuning-3agreements-only"): # poetry-weighted-finetuning 로 변경
    # version_0, version_1, ... version_50 경로를 모두 찾아 리스트업
    version_dirs = glob.glob(os.path.join(base_dir, "version_*"))
    # 버전 숫자를 기준으로 정렬
    version_dirs.sort(key=lambda x: int(x.split("_")[-1]))
    if not version_dirs:
        raise FileNotFoundError(f"No version_* directories found under '{base_dir}'")
    # 가장 마지막(숫자가 가장 큰) 버전 경로 반환
    return version_dirs[-1]

def get_latest_checkpoint_poetry(version_dir):
    ckpt_dir = os.path.join(version_dir, "checkpoints")
    ckpt_list = glob.glob(os.path.join(ckpt_dir, "*.ckpt"))
    ckpt_list.sort()  # 파일명 기준 정렬
    if not ckpt_list:
        raise FileNotFoundError(f"No .ckpt found under '{ckpt_dir}'")
    # 가장 마지막 파일(정렬 기준)
    return ckpt_list[-1]

In [19]:
latest_version_dir_poetry = get_latest_version_dir_poetry("./lightning_logs/poetry-finetuning-3agreements-only") # poetry-weighted-finetuning 로 변경
best_ckpt_path_poetry = get_latest_checkpoint_poetry(latest_version_dir_poetry)
print("Best checkpoint path (Poetry Weighted Finetuned):", best_ckpt_path_poetry) # poetry-weighted-finetuning 로 변경

best_model_poetry = PoetryTagger.load_from_checkpoint(best_ckpt_path_poetry)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
best_model_poetry.to(device)
best_model_poetry.eval()
best_model_poetry.freeze()

Best checkpoint path (Poetry Weighted Finetuned): ./lightning_logs/poetry-finetuning-3agreements-only/version_0/checkpoints/epoch9-val_loss0.2566.ckpt


In [20]:
best_model_poetry = PoetryTagger.load_from_checkpoint(best_ckpt_path_poetry)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
best_model_poetry.to(device)
best_model_poetry.eval()
best_model_poetry.freeze()

In [21]:
encoding = tokenizer(
    sample_text,
    max_length=512,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)

with torch.no_grad():
    # 입력 텐서 또한 같은 device로 이동
    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)

    # forward
    _, predictions = best_model_poetry(input_ids, attention_mask) # best_model_poetry 사용

# 추론 결과를 CPU로 가져와 numpy로 변환
predictions = predictions.flatten().cpu().numpy()

print("\n[Sample Inference 결과]")
for label_name, score in zip(LABELS, predictions): # LABELS 변수 확인
    if score > THRESHOLD:
        print(f"{label_name} : {score:.3f}")

# [Sample Inference 결과]
# 불안/걱정 : 0.336
# 슬픔 : 0.311
# 안타까움/실망 : 0.344


[Sample Inference 결과]
공포/무서움 : 0.434
놀람 : 0.308
당황/난처 : 0.513
부담/안_내킴 : 0.399
불안/걱정 : 0.549
불평/불만 : 0.302
비장함 : 0.338
서러움 : 0.356
슬픔 : 0.416
신기함/관심 : 0.309
안타까움/실망 : 0.308
의심/불신 : 0.305
힘듦/지침 : 0.342


In [22]:
encoding = tokenizer(
    sample_text,
    max_length=512,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)

with torch.no_grad():
    # 입력 텐서 또한 같은 device로 이동
    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)

    # forward
    _, predictions = best_model_poetry(input_ids, attention_mask)  # best_model_poetry 사용

# 추론 결과를 CPU로 가져와 numpy로 변환
predictions = predictions.flatten().cpu().numpy()

# 결과를 딕셔너리로 저장 (숫자값으로 변환)
result_dict = {
    label_name: float(round(score, 3))  # np.float32 -> float 변환
    for label_name, score in zip(LABELS, predictions)
    if score > THRESHOLD
}

# 결과 출력
print("\n[Sample Inference 결과]")
print(result_dict)

# 예시 출력
# {'불안/걱정': 0.336, '슬픔': 0.311}


[Sample Inference 결과]
{'공포/무서움': 0.4339999854564667, '놀람': 0.30799999833106995, '당황/난처': 0.5130000114440918, '부담/안_내킴': 0.39899998903274536, '불안/걱정': 0.5490000247955322, '불평/불만': 0.3019999861717224, '비장함': 0.33799999952316284, '서러움': 0.35600000619888306, '슬픔': 0.41600000858306885, '신기함/관심': 0.3089999854564667, '안타까움/실망': 0.30799999833106995, '의심/불신': 0.3050000071525574, '힘듦/지침': 0.34200000762939453}


### Ollama Llama + KPoEM classification Model LangChain

In [23]:
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [24]:
# 3. Ollama Llama 모델 로드
# Ollama 서버가 로컬에서 실행 중이어야 합니다.
# 모델 이름은 'ollama run llama3' 등으로 다운로드한 모델 이름과 일치해야 합니다.
llm = Ollama(model="llama3.2")

/var/folders/cd/mj8pxhhd6ml7kpdhggf19l_80000gn/T/ipykernel_69717/1217402929.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.2")


In [25]:
# 감정 분류 결과 예시 (임의의 함수 또는 모델로 대체)
def classify_emotion(sample_text):
    # 예: {'슬픔': 0.6, '기쁨': 0.3, '분노': 0.1}
    encoding = tokenizer(
    sample_text,
    max_length=512,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
    )

    with torch.no_grad():
    # 입력 텐서 또한 같은 device로 이동
        input_ids = encoding["input_ids"].to(device)
        attention_mask = encoding["attention_mask"].to(device)

    # forward
        _, predictions = best_model_poetry(input_ids, attention_mask)  # best_model_poetry 사용

# 추론 결과를 CPU로 가져와 numpy로 변환
    predictions = predictions.flatten().cpu().numpy()

# 결과를 딕셔너리로 저장 (숫자값으로 변환)
    result_dict = {
        label_name: float(round(score, 3))  # np.float32 -> float 변환
        for label_name, score in zip(LABELS, predictions)
        if score > THRESHOLD
    }

# 결과 출력
    print("\n[Sample Inference 결과]")
    print(result_dict)

# 예시 출력
# {'불안/걱정': 0.336, '슬픔': 0.311}
    return result_dict

In [26]:
# 2️⃣ 감정 기반 프롬프트 생성
def generate_prompt(emotion_scores):
    top_emotion = max(emotion_scores, key=emotion_scores.get)
    return f"""당신은 감정이 섬세한 한국 현대시 작가입니다.
'{top_emotion}'의 감정을 중심으로 짧은 시를 한 편 창작해 주세요."""

In [27]:
# 4️⃣ LangChain 프롬프트 템플릿 구성
prompt = PromptTemplate.from_template("{emotion_prompt}")
chain = LLMChain(llm=llm, prompt=prompt)

/var/folders/cd/mj8pxhhd6ml7kpdhggf19l_80000gn/T/ipykernel_69717/1438890832.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [28]:
# 5️⃣ 전체 흐름 함수
def emotion_to_poetry(user_input):
    emotion_scores = classify_emotion(user_input)
    emotion_prompt = generate_prompt(emotion_scores)
    poem = chain.run(emotion_prompt=emotion_prompt)
    print(emotion_scores) # 감정 점수 출력
    return poem

In [52]:
# 6️⃣ 테스트
input_text = """하루 종일 지친 몸으로만 떠돌다가
땅에 떨어져 죽지 못한
햇빛들은 줄지어 어디로 가는 걸까

웅성웅성 가장 근심스러운 색깔로 서행하며
이미 어둠이 깔리는 소각장으로 몰려들어
몇 점 폐휴지로 타들어가는 오루 6시의 참혹한 형량
단 한 번 후회도 용서하지 않는 무서운 시간
바람은 긴 채찍을 휘둘러
살아서 빛나는 온갖 상징을 몰아내고 있다.

도시는 곧 활자들이 일제히 빠져 달아나
속도 없이 페이지를 펄럭이는 텅 빈 한 권 책이 되리라.
승부를 알 수 없는 하루와의 싸움에서
우리는 패배했을까. 오늘도 물어보는 사소한 물음은
그러나 우리의 일생을 텅텅 흔드는 것.

오후 6시의 소각장 위로 말없이
검은 연기가 우산처럼 펼쳐지고
이젠 우리들의 차례였다.
두렵지 않은가.
밤이면 그림자를 빼앗겨 누구나 아득한 혼자였다.

문득 거리를 빠르게 스쳐가는 일상의 공포
보여다오. 지금까지 무엇을 했는가 살아 있는 그대여
오후 6시 우리들 이마에도 아, 붉은 노을이 떴다.

그러면 우리는 어디로 가지?
아직도 펄펄 살아 있는 우리는 이제 각자 어디로 가지?
""" # 기형도 - 노을
generated_poem = emotion_to_poetry(input_text)

print("🎴 생성된 시:\n")
print(generated_poem)


[Sample Inference 결과]
{'공포/무서움': 0.4339999854564667, '놀람': 0.30799999833106995, '당황/난처': 0.5130000114440918, '부담/안_내킴': 0.39899998903274536, '불안/걱정': 0.5490000247955322, '불평/불만': 0.3019999861717224, '비장함': 0.33799999952316284, '서러움': 0.35600000619888306, '슬픔': 0.41600000858306885, '신기함/관심': 0.3089999854564667, '안타까움/실망': 0.30799999833106995, '의심/불신': 0.3050000071525574, '힘듦/지침': 0.34200000762939453}
🎴 생성된 시:

걱정의 깊은 물결
안정 밖에 있겠나?
사랑의 말로 못할 깊은 불안에 깨어질까
근심스럽다. 깊숙한梦과 근심이existence는 어울리기 어렵지만, 

어떨 수 있을지
꿈과 근심이
 deep-seated bulan의 물결을 달가하는지?

시를 지어 주는 감정의 분위기는 '걱정'과 '불안'을 담아 낸 '深層 불안'으로 표현되었습니다. 시는 " 안정 밖에 있겠나"로 시작하여 "사랑의 말로 못할 깊은 불안에 깨어질까"로 이어져지며, 이 두 문장을 연결하여 "근심스럽다. 깊숙한梦과 근심이"라는 문장을 추가하였습니다. 시는 " deep-seated bulan"으로 표현되었습니다.


## 벡터 DB 연결

In [30]:
texts = df["본문"].dropna().astype(str).tolist()
texts

['죽는 날까지 하늘을 우러러',
 '한 점 부끄럼이 없기를,',
 '잎새에 이는 바람에도',
 '나는 괴로워했다.',
 '별을 노래하는 마음으로',
 '모든 죽어가는 것을 사랑해야지',
 '그리고 나한테 주어진 길을',
 '걸어가야겠다.',
 '오늘 밤에도 별이 바람에 스치운다.',
 '산모퉁이를 돌아 논가 외딴 우물을 홀로 찾아가선 가만히 들여다봅니다.',
 '우물 속에는 달이 밝고 구름이 흐르고 하늘이 펼치고 파아란 바람이 불고 가을이 있습니다.',
 '그리고 한 사나이가 있습니다.',
 '어쩐지 그 사나이가 미워져 돌아갑니다.',
 '돌아가다 생각하니 그 사나이가 가엾어집니다.',
 '도로 가 들여다보니 사나이는 그대로 있습니다.',
 '다시 그 사나이가 미워져 돌아갑니다.',
 '돌아가다 생각하니 그 사나이가 그리워집니다.',
 '우물 속에는 달이 밝고 구름이 흐르고 하늘이 펼치고 파아란 바람이 불고 가을이 있고 추억처럼 사나이가 있습니다.',
 '쫓아오든 햇빛인데',
 '지금 교회당 꼭대기',
 '십자가에 걸리었습니다.',
 '첨탑(尖塔)이 저렇게도 높은데',
 '어떻게 올라갈 수 있을까요.',
 '종소리도 들려 오지 않는데',
 '휘파람이나 불며 서성거리다가',
 '괴로웠던 사나이',
 '행복한 예수 그리스도에게',
 '처럼',
 '십자가가 허락된다면',
 '모가지를 드리우고',
 '꽃처럼 피어나는 피를',
 '어두워 가는 하늘 밑에',
 '조용히 흘리겠습니다.',
 '바람이 어디로부터 불어 와',
 '어디로 불려 가는 것일까',
 '바람이 부는데',
 '내 괴로움에는 이유가 없다.',
 '내 괴로움에는 이유가 없을까',
 '단 한 여자를 사랑한 일도 없다.',
 '시대를 슬퍼한 일도 없다.',
 '바람이 자꼬 부는데',
 '내 발이 반석 우에 섰다.',
 '강물이 자꼬 흐르는데',
 '내 발이 언덕 우에 섰다.',
 '고향에 돌아온 날 밤에',
 '내 백골(白骨)이 따라와 한방에 누웠다.',
 '어둔 방은 우주로 통하고',
 '하늘에선가 소리처럼

In [32]:
from langchain.docstore.document import Document

In [33]:
# 2️⃣ 문장들을 Document 형태로 변환
documents = [Document(page_content=text) for text in texts]

In [34]:
documents

[Document(metadata={}, page_content='죽는 날까지 하늘을 우러러'),
 Document(metadata={}, page_content='한 점 부끄럼이 없기를,'),
 Document(metadata={}, page_content='잎새에 이는 바람에도'),
 Document(metadata={}, page_content='나는 괴로워했다.'),
 Document(metadata={}, page_content='별을 노래하는 마음으로'),
 Document(metadata={}, page_content='모든 죽어가는 것을 사랑해야지'),
 Document(metadata={}, page_content='그리고 나한테 주어진 길을'),
 Document(metadata={}, page_content='걸어가야겠다.'),
 Document(metadata={}, page_content='오늘 밤에도 별이 바람에 스치운다.'),
 Document(metadata={}, page_content='산모퉁이를 돌아 논가 외딴 우물을 홀로 찾아가선 가만히 들여다봅니다.'),
 Document(metadata={}, page_content='우물 속에는 달이 밝고 구름이 흐르고 하늘이 펼치고 파아란 바람이 불고 가을이 있습니다.'),
 Document(metadata={}, page_content='그리고 한 사나이가 있습니다.'),
 Document(metadata={}, page_content='어쩐지 그 사나이가 미워져 돌아갑니다.'),
 Document(metadata={}, page_content='돌아가다 생각하니 그 사나이가 가엾어집니다.'),
 Document(metadata={}, page_content='도로 가 들여다보니 사나이는 그대로 있습니다.'),
 Document(metadata={}, page_content='다시 그 사나이가 미워져 돌아갑니다.'),
 Document(metadata={}, page_content=

In [36]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

In [38]:
! pip install sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.7/345.7 kB 9.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [39]:

# 3️⃣ 벡터 임베딩 모델 로딩 (한국어 지원하는 모델 권장)
embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-sts")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.44k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [41]:
! pip install faiss-cpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 11.1 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [42]:
# 4️⃣ FAISS VectorDB 생성
vectorstore = FAISS.from_documents(documents, embedding_model)

In [46]:
# 2️⃣ 감정 기반 프롬프트 생성
def generate_prompt_withVector(emotion_scores, context_snippets):
    top_emotion = max(emotion_scores, key=emotion_scores.get)
    return f"""당신은 감정이 섬세한 한국 현대시 작가입니다.
'{top_emotion}'의 감정을 중심으로 짧은 시를 한 편 창작해 주세요.
다음은 감정의 분위기를 도와줄 참고 문장입니다:
{context_snippets}

이 문장들을 참고하여 시를 지어 주세요.
"""

In [47]:

# 8️⃣ 전체 체인
def emotion_to_poetry(user_input):
    scores = classify_emotion(user_input)
    top_emotion = max(scores, key=scores.get)

    # 관련 시구 검색
    context_snippets = vectorstore.similarity_search(top_emotion, k=5)

    # 프롬프트 생성
    full_prompt = generate_prompt_withVector(scores, context_snippets)

    # LangChain Prompt + LLM 실행
    prompt = PromptTemplate.from_template("{emotion_prompt}")
    chain = LLMChain(llm=llm, prompt=prompt)
    result = chain.run(emotion_prompt=full_prompt)

    return result

In [53]:
# 9️⃣ 테스트 실행
user_text = """하루 종일 지친 몸으로만 떠돌다가
땅에 떨어져 죽지 못한
햇빛들은 줄지어 어디로 가는 걸까


웅성웅성 가장 근심스러운 색깔로 서행하며
이미 어둠이 깔리는 소각장으로 몰려들어
몇 점 폐휴지로 타들어가는 오루 6시의 참혹한 형량
단 한 번 후회도 용서하지 않는 무서운 시간
바람은 긴 채찍을 휘둘러
살아서 빛나는 온갖 상징을 몰아내고 있다.


도시는 곧 활자들이 일제히 빠져 달아나
속도 없이 페이지를 펄럭이는 텅 빈 한 권 책이 되리라.
승부를 알 수 없는 하루와의 싸움에서
우리는 패배했을까. 오늘도 물어보는 사소한 물음은
그러나 우리의 일생을 텅텅 흔드는 것.


오후 6시의 소각장 위로 말없이
검은 연기가 우산처럼 펼쳐지고
이젠 우리들의 차례였다.
두렵지 않은가.
밤이면 그림자를 빼앗겨 누구나 아득한 혼자였다.


문득 거리를 빠르게 스쳐가는 일상의 공포
보여다오. 지금까지 무엇을 했는가 살아 있는 그대여
오후 6시 우리들 이마에도 아, 붉은 노을이 떴다.


그러면 우리는 어디로 가지?
아직도 펄펄 살아 있는 우리는 이제 각자 어디로 가지?
""" # 기형도 - 노을
generated_poem = emotion_to_poetry(user_text)

print("🎴 생성된 시:\n", generated_poem)


[Sample Inference 결과]
{'공포/무서움': 0.4350000023841858, '놀람': 0.30799999833106995, '당황/난처': 0.5139999985694885, '부담/안_내킴': 0.39899998903274536, '불안/걱정': 0.5479999780654907, '불평/불만': 0.30300000309944153, '비장함': 0.33899998664855957, '서러움': 0.35100001096725464, '슬픔': 0.414000004529953, '신기함/관심': 0.3109999895095825, '안타까움/실망': 0.30799999833106995, '의심/불신': 0.30799999833106995, '힘듦/지침': 0.33899998664855957}
🎴 생성된 시:
 아름다운 밤을 지내도, 

걱정이든, 깨어질까 근심스럽다. 

사랑의 말로 못할 깊은 불안에, 

안정 밖에 있겠나. 

꿈과 근심이 있거든, 

그리고 그 밤에는 무언가의 불안한 미리미리

저는 내일을 두려워한다.

지금은 어둠에 포용된다.

그 nighttime에는 깊은 불안이 찾아오고,

어찌할 수 있는 안정감을 잃었다.

어떤夜에 깊은 근심이 지속되기 시작했다. 

그 밤에는 내일과Tomorrow가의 불안감을 받았다.

그리고 그밖의 모든 불안감을 다 가슴에 안고

어떤nightingale에 내성스러운 우울함이 들어간다

그 nighttime에서, 

저는 어두운 nightside가의 불안에 잠들어 있었다.
